# Video Classifier Using CNN and RNN
#!dir
##### Credit to Aarohi at https://www.youtube.com/watch?v=ezjnySXqdTo&list=WL&index=22&t=867s

In [ ]:
#code to use google drive
#Skip this if not using google drive/colab
#All of the paths and code below assume the usage of google drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/Foo/dataset/train')

label_types = os.listdir('/content/drive/MyDrive/Foo/dataset/train')
print (label_types)  

['PS', 'PE']


# Preparing Training Data

In [ ]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/Foo/dataset/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/Foo/dataset/train' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

#Creating and building a dataframe


  tag                                         video_name
0  PS  /content/drive/MyDrive/Foo/dataset/train/PS/13...
1  PS  /content/drive/MyDrive/Foo/dataset/train/PS/10...
2  PS  /content/drive/MyDrive/Foo/dataset/train/PS/15...
3  PS  /content/drive/MyDrive/Foo/dataset/train/PS/1.mp4
4  PS  /content/drive/MyDrive/Foo/dataset/train/PS/14...
    tag                                         video_name
159  PE  /content/drive/MyDrive/Foo/dataset/train/PE/95...
160  PE  /content/drive/MyDrive/Foo/dataset/train/PE/91...
161  PE  /content/drive/MyDrive/Foo/dataset/train/PE/98...
162  PE  /content/drive/MyDrive/Foo/dataset/train/PE/99...
163  PE  /content/drive/MyDrive/Foo/dataset/train/PE/97...


In [ ]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')
#creating train csv file

# Preparing Test Data

In [ ]:
#reading Test data
dataset_path = os.listdir('/content/drive/MyDrive/Foo/dataset/test')
print(dataset_path)

room_types = os.listdir('/content/drive/MyDrive/Foo/dataset/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/Foo/dataset/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/Foo/dataset/test' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['PS', 'PE']
Types of activities found:  2
  tag                                         video_name
0  PS  /content/drive/MyDrive/Foo/dataset/test/PS/10.mp4
1  PS  /content/drive/MyDrive/Foo/dataset/test/PS/12.mp4
2  PS  /content/drive/MyDrive/Foo/dataset/test/PS/27.mp4
3  PS  /content/drive/MyDrive/Foo/dataset/test/PS/22.mp4
4  PS  /content/drive/MyDrive/Foo/dataset/test/PS/24.mp4
   tag                                         video_name
75  PE  /content/drive/MyDrive/Foo/dataset/test/PE/66.mp4
76  PE  /content/drive/MyDrive/Foo/dataset/test/PE/68.mp4
77  PE  /content/drive/MyDrive/Foo/dataset/test/PE/72.mp4
78  PE  /content/drive/MyDrive/Foo/dataset/test/PE/82.mp4
79  PE  /content/drive/MyDrive/Foo/dataset/test/PE/96.mp4


In [ ]:
!pip install git+https://github.com/tensorflow/docs
#Need to install this if I want to use it for first time.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-aezc1m1s
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-aezc1m1s
  Resolved https://github.com/tensorflow/docs to commit a81ea51cef0734c602915ba3a5d4f4c755243818
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=182612 sha256=52dbb5892b5af9c14e99d62a46405933509e7c2446695a6376b72d497dcc2fc1
  Stored in directory: /tmp/pip-ephem-wheel-cache-fahprfi8/wheels/fc/f8/3b/5d21409a59cb1be9b1ade11f682039ced75b84de9dd6a0c8de
Successfully built tensorflow-docs


In [ ]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
#Required modules

In [ ]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

# Data preparation

In [ ]:
#Reading train.csv and test.csv file and seeing how many of each we have
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 164
Total videos for testing: 80


,Unnamed: 0,video_name,tag
48,48,/content/drive/MyDrive/Foo/dataset/train/PS/71...,PS
39,39,/content/drive/MyDrive/Foo/dataset/train/PS/61...,PS
136,136,/content/drive/MyDrive/Foo/dataset/train/PE/8.mp4,PE
63,63,/content/drive/MyDrive/Foo/dataset/train/PS/89...,PS
25,25,/content/drive/MyDrive/Foo/dataset/train/PS/44...,PS
108,108,/content/drive/MyDrive/Foo/dataset/train/PE/38...,PE
160,160,/content/drive/MyDrive/Foo/dataset/train/PE/91...,PE
137,137,/content/drive/MyDrive/Foo/dataset/train/PE/79...,PE
65,65,/content/drive/MyDrive/Foo/dataset/train/PS/95...,PS
129,129,/content/drive/MyDrive/Foo/dataset/train/PE/64...,PE


# Feed the videos to a network:


In [ ]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# Using Opencv video capture method will capture the frames from the videos and feed it
# Then, it will extract frames until max is reached. And we also use padding, so we put some extra stuff in ones with little stuff.
# 
IMG_SIZE = 224

#Cropping center square
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

#loading video to the algorithm
#Read the frames of the video
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]] #Storing resized frames
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

   ### Feature Extraction

In [ ]:
#Extracting features for classification
#InceptionV3 is the model we're using to extract - This is the CNN part
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 3s 0us/step


### Label Encoding
StringLookup layer encode the class labels as integers.

In [ ]:
#We have to convert the strings into numbers and we're doing that here!
#ex) 0 = dancing, 1 = others... etc.
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['PE', 'PS']


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

Finally, we can put all the pieces together to create our data processing utility.

In [ ]:
#print(train_data[0].shape)
#train_data[0]

In [ ]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters

1/1 [==============================] - 0s 172ms/step
Frame features in train set: (164, 20, 2048)
Frame masks in train set: (164, 20)
train_labels in train set: (164, 1)
test_labels in train set: (80, 1)


# The sequence model
Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
3/4 [=====================>........] - ETA: 0s - loss: 0.6350 - accuracy: 0.6146 
Epoch 1: val_loss improved from inf to 0.45291, saving model to ./tmp/video_classifier
4/4 [==============================] - 11s 923ms/step - loss: 0.6131 - accuracy: 0.6404 - val_loss: 0.4529 - val_accuracy: 1.0000
Epoch 2/30
3/4 [=====================>........] - ETA: 0s - loss: 0.3697 - accuracy: 0.8958
Epoch 2: val_loss did not improve from 0.45291
4/4 [==============================] - 0s 59ms/step - loss: 0.3736 - accuracy: 0.8684 - val_loss: 0.4668 - val_accuracy: 0.9000
Epoch 3/30
3/4 [=====================>........] - ETA: 0s - loss: 0.2509 - accuracy: 0.9375
Epoch 3: val_loss improved from 0.45291 to 0.33642, saving model to ./tmp/video_classifier
4/4 [==============================] - 0s 66ms/step - loss: 0.2508 - accuracy: 0.9474 - val_loss: 0.3364 - val_accuracy: 1.0000
Epoch 4/30
3/4 [=====================>........] - ETA: 0s - loss: 0.2440 - accuracy: 0.9896
Epoch 4: val_loss im

# Inference

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: /content/drive/MyDrive/Foo/dataset/test/PE/109.mp4
1/1 [==============================] - 2s 2s/step
  PE: 95.29%
  PS:  4.71%
